In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.optim import Adam
import datetime
import torch
import os
from torch.utils.tensorboard import SummaryWriter

In [2]:
device = torch.device('cuda:1')

if torch.cuda.is_available():
    print("Available CUDA devices:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")

Available CUDA devices: 2
Device 0: NVIDIA GeForce RTX 4090
Device 1: NVIDIA GeForce RTX 4090


In [23]:
# inputdir='../data/'
# # Load data
# train_data = pd.read_csv(inputdir+'loop_sensor_train.csv')
# baseline_data = pd.read_csv(inputdir+'loop_sensor_test_baseline.csv')
# test_data_x = pd.read_csv(inputdir+'loop_sensor_test_x.csv')

# # Prepare test data
# test_data_x['t_1h'] = pd.to_datetime(test_data_x['t_1h'])
# ref_time = pd.Timestamp('2022-01-01 00:00:00')
# test_data_x['t_1h'] = (test_data_x['t_1h'] - ref_time).dt.total_seconds() / 3600.0

# # Convert test data to tensor
# test_features = torch.tensor(test_data_x[['iu_ac', 't_1h', 'etat_barre']].values).float().to(device)
# # Convert time to datetime
# train_data['t_1h'] = pd.to_datetime(train_data['t_1h'])
# test_data_x['t_1h'] = pd.to_datetime(test_data_x['t_1h'])

# # Preprocess and split data for year 2023
# def preprocess_data(data):
#     train = data
#     for name, group in data[data['t_1h'].dt.year == 2023].groupby(pd.Grouper(key='t_1h', freq='29h')):
#         train = pd.concat([train, group.iloc[:24]])
#     return train

# train     = preprocess_data(train_data)
# eval_data = baseline_data
# print(train.shape, eval_data.shape)

(23139649, 4) (439298, 2)


In [12]:
inputdir = '../data/'
# Load data
train_data = pd.read_csv(f'{inputdir}loop_sensor_train.csv')
eval_data = pd.read_csv(f'{inputdir}loop_sensor_eval.csv')
test_data_x = pd.read_csv(f'{inputdir}loop_sensor_test_x.csv')

def to_datetime(data):
    data['t_1h'] = pd.to_datetime(data['t_1h'])

to_datetime(train_data)
to_datetime(eval_data)
to_datetime(test_data_x)

# Function to prepare training data for 2023
def prepare_2023_data(data):
    processed_data = data
    # Split data for year 2023 into 24-hour frames and the following 5-hour as test
    for name, group in data[data['t_1h'].dt.year == 2023].groupby(pd.Grouper(key='t_1h', freq='29h')):
        processed_data = pd.concat([processed_data, group.iloc[:24]])
    return processed_data
train_data = prepare_2023_data(train_data)


# Debugging the data shapes
print(f"Train data shape: {train_data.shape}")
print(f"Evaluation data shape: {eval_data.shape}")
print(f"Test data shape: {test_data_x.shape}")


Train data shape: (23139649, 4)
Evaluation data shape: (428193, 5)
Test data shape: (439298, 4)


In [13]:
# Function to encode categorical data into one-hot vectors
def encode_categorical(data, column_name, num_classes):
    categories = torch.tensor(data[column_name].values)
    return torch.nn.functional.one_hot(categories, num_classes=num_classes).float()

def data_to_tensor(data, train=1):
    print("Number of rows in data:", len(data))
    if 'id' in data.columns:
        data.drop('id', axis=1, inplace=True)
    # Convert 't_1h' from datetime to a float representing hours since the start of the dataset
    ref_time = pd.Timestamp('2022-01-01 00:00:00')  # Adjust the reference time as needed
    data['t_1h'] = (data['t_1h'] - ref_time).dt.total_seconds() / 3600.0
    # One-hot encode 'etat_barre'
    etat_barre_encoded = encode_categorical(data, 'etat_barre', 4)
    # Convert to tensors
    iu_ac = torch.tensor(data['iu_ac'].values).unsqueeze(1).float()
    t_1h = torch.tensor(data['t_1h'].values).unsqueeze(1).float()
    # Combine features into a single tensor
    features = torch.cat((iu_ac, t_1h, etat_barre_encoded), dim=1)
    
    if train:
        targets = torch.tensor(data['q'].values).float()
        return TensorDataset(features, targets)
    return TensorDataset(features)

# Assuming 'train', 'eval_data', and 'test_data_x' are already loaded and preprocessed
train_dataset = data_to_tensor(train_data)
eval_dataset = data_to_tensor(eval_data)
test_dataset_x = data_to_tensor(test_data_x, train=0)
# Convert test data to tensor
# test_features = torch.tensor(test_data_x[['iu_ac', 't_1h', 'etat_barre']].values).float().to(device)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=2048, shuffle=False)
test_loader = DataLoader(test_dataset_x, batch_size=2048, shuffle=False)


Number of rows in data: 23139649
Number of rows in data: 428193
Number of rows in data: 439298


In [5]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout=0.5):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.regressor = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        out = self.regressor(hn.squeeze(0))
        return out

# Example with more hidden units and layers
model = LSTMModel(input_dim=6, hidden_dim=500, num_layers=3, dropout=0.5)
model.to(device)

# Loss and optimizer
criterion = nn.L1Loss()
optimizer = Adam(model.parameters(), lr=0.001)

In [7]:
def train_model(model, train_loader, criterion, optimizer, num_epochs, log_interval=10, checkpoint_dir='./logs'):
    # Move model to GPU
    device = torch.device('cuda:1')
    model.to(device)
    
    # Initialize TensorBoard writer
    writer = SummaryWriter()
    start_epoch = 1
    flag = True
    checkpoint_path = os.path.join(checkpoint_dir, 'checkpoint.pth')  # specify the filename for checkpoint
    
    # Ensure checkpoint directory exists
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    # Check if checkpoint exists
    if os.path.exists(checkpoint_path):
        print("Loading checkpoint...")
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1  # start from next epoch
        
    model.train()
    for epoch in range(start_epoch, num_epochs + start_epoch):
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            if flag:
                print(next(model.parameters()).is_cuda)  
                print(inputs.is_cuda)                    
                flag = False
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            # Log loss to TensorBoard
            if batch_idx % log_interval == 0:
                writer.add_scalar('Loss/train', loss.item(), epoch * len(train_loader) + batch_idx)
            
            print(f'Epoch {epoch}, Batch {batch_idx+1}, Loss: {loss.item()}')
            # Save checkpoint periodically
            if batch_idx % 100 == 0:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss.item(),
                }, checkpoint_path)
    # Close the TensorBoard writer
    writer.close()

train_model(model, train_loader, criterion, optimizer, num_epochs=1)

True
True
Epoch 1, Batch 1, Loss: 611.5944213867188
Epoch 1, Batch 2, Loss: 663.2142944335938
Epoch 1, Batch 3, Loss: 635.466064453125
Epoch 1, Batch 4, Loss: 648.3395385742188
Epoch 1, Batch 5, Loss: 668.6637573242188
Epoch 1, Batch 6, Loss: 647.774169921875
Epoch 1, Batch 7, Loss: 615.0032958984375
Epoch 1, Batch 8, Loss: 660.638916015625
Epoch 1, Batch 9, Loss: 686.8062744140625
Epoch 1, Batch 10, Loss: 666.5144653320312
Epoch 1, Batch 11, Loss: 677.5919189453125
Epoch 1, Batch 12, Loss: 609.7320556640625
Epoch 1, Batch 13, Loss: 674.9031372070312
Epoch 1, Batch 14, Loss: 689.4656982421875
Epoch 1, Batch 15, Loss: 645.3823852539062
Epoch 1, Batch 16, Loss: 616.0963134765625
Epoch 1, Batch 17, Loss: 622.2770385742188
Epoch 1, Batch 18, Loss: 706.7965698242188
Epoch 1, Batch 19, Loss: 655.4170532226562
Epoch 1, Batch 20, Loss: 629.0782470703125
Epoch 1, Batch 21, Loss: 625.9249877929688
Epoch 1, Batch 22, Loss: 643.7152099609375
Epoch 1, Batch 23, Loss: 660.04541015625
Epoch 1, Batch 

/media/ssd4t/anaconda3/envs/traffic/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([1345])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch 1, Batch 11299, Loss: 524.3890991210938
Epoch 2, Batch 1, Loss: 531.7263793945312
Epoch 2, Batch 2, Loss: 512.444091796875
Epoch 2, Batch 3, Loss: 512.2116088867188
Epoch 2, Batch 4, Loss: 548.04052734375
Epoch 2, Batch 5, Loss: 508.18939208984375
Epoch 2, Batch 6, Loss: 525.157470703125
Epoch 2, Batch 7, Loss: 531.6629638671875
Epoch 2, Batch 8, Loss: 491.44842529296875
Epoch 2, Batch 9, Loss: 515.99072265625
Epoch 2, Batch 10, Loss: 524.7669677734375
Epoch 2, Batch 11, Loss: 505.36773681640625
Epoch 2, Batch 12, Loss: 511.9063415527344
Epoch 2, Batch 13, Loss: 532.5601806640625
Epoch 2, Batch 14, Loss: 499.329833984375
Epoch 2, Batch 15, Loss: 546.8385620117188
Epoch 2, Batch 16, Loss: 507.1145935058594
Epoch 2, Batch 17, Loss: 545.7750854492188
Epoch 2, Batch 18, Loss: 511.70770263671875
Epoch 2, Batch 19, Loss: 542.70458984375
Epoch 2, Batch 20, Loss: 478.3123474121094
Epoch 2, Batch 21, Loss: 512.4847412109375
Epoch 2, Batch 22, Loss: 533.9347534179688
Epoch 2, Batch 23, Los

In [8]:
# Evaluation
def evaluate_model(model, eval_loader):
    model.eval()
    with torch.no_grad():
        total_loss = 0
        for inputs, targets in eval_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
        print(f'Evaluation Loss: {total_loss / len(eval_loader)}')

evaluate_model(model, eval_loader)


Evaluation Loss: 517.3623548598516


/media/ssd4t/anaconda3/envs/traffic/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([161])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


In [24]:
# Predict
model.eval()
predictions = []
with torch.no_grad():
    i = 1
    for features, in test_loader:
        i+=1
        features = features.to(device)
        outputs = model(features)
        predictions.extend(outputs.cpu().numpy().flatten())
        print(predictions)
        print(i, features.shape , len(predictions), len(outputs.cpu().numpy().flatten()))
# Create a new DataFrame with only predictions, set index starting from 1
predictions_df = pd.DataFrame(predictions, columns=['predicted_q'])
predictions_df.index = predictions_df.index + 1  # Adjust index to start from 1

print(len(predictions_df))
# Save the predictions to a CSV file
predictions_df.to_csv(inputdir+'predictions_only.csv', index_label='ID')

print("Predictions saved to 'predictions_only.csv', with IDs starting from 1.")

[306.98007, 345.0207, 345.0365]
2 torch.Size([2048, 6]) 3 3
[306.98007, 345.0207, 345.0365, 306.90067, 345.0207, 345.0365]
3 torch.Size([2048, 6]) 6 3
[306.98007, 345.0207, 345.0365, 306.90067, 345.0207, 345.0365, 306.874, 345.0207, 345.0365]
4 torch.Size([2048, 6]) 9 3
[306.98007, 345.0207, 345.0365, 306.90067, 345.0207, 345.0365, 306.874, 345.0207, 345.0365, 306.89334, 345.0207, 345.0365]
5 torch.Size([2048, 6]) 12 3
[306.98007, 345.0207, 345.0365, 306.90067, 345.0207, 345.0365, 306.874, 345.0207, 345.0365, 306.89334, 345.0207, 345.0365, 306.86362, 345.0207, 345.0365]
6 torch.Size([2048, 6]) 15 3
[306.98007, 345.0207, 345.0365, 306.90067, 345.0207, 345.0365, 306.874, 345.0207, 345.0365, 306.89334, 345.0207, 345.0365, 306.86362, 345.0207, 345.0365, 306.84317, 345.0207, 345.0365]
7 torch.Size([2048, 6]) 18 3
[306.98007, 345.0207, 345.0365, 306.90067, 345.0207, 345.0365, 306.874, 345.0207, 345.0365, 306.89334, 345.0207, 345.0365, 306.86362, 345.0207, 345.0365, 306.84317, 345.0207, 345.0